In [1]:
# 利用SMO算法求解7.95-7.97
# 选取前两类做实验
# 选择前80%作为训练集，后20%作为测试集

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [2]:
# data
def create_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df['label'] = iris.target

    df.columns = ['sepal length', 'sepal width', 'petal length', 'petal width', 'label']
    print(df)
    data = np.array(df.iloc[:100, :])
    #print(data)
    #print(len(data))
    for i in range(len(data)):
        if data[i,-1] == 0:
            data[i,-1] = -1
    print(data)
    return data[:, :4], data[:, -1]

In [3]:
X, y = create_data()
print(X.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

     sepal length  sepal width  petal length  petal width  label
0             5.1          3.5           1.4          0.2      0
1             4.9          3.0           1.4          0.2      0
2             4.7          3.2           1.3          0.2      0
3             4.6          3.1           1.5          0.2      0
4             5.0          3.6           1.4          0.2      0
..            ...          ...           ...          ...    ...
145           6.7          3.0           5.2          2.3      2
146           6.3          2.5           5.0          1.9      2
147           6.5          3.0           5.2          2.0      2
148           6.2          3.4           5.4          2.3      2
149           5.9          3.0           5.1          1.8      2

[150 rows x 5 columns]
[[ 5.1  3.5  1.4  0.2 -1. ]
 [ 4.9  3.   1.4  0.2 -1. ]
 [ 4.7  3.2  1.3  0.2 -1. ]
 [ 4.6  3.1  1.5  0.2 -1. ]
 [ 5.   3.6  1.4  0.2 -1. ]
 [ 5.4  3.9  1.7  0.4 -1. ]
 [ 4.6  3.4  1.4  0.3 -1. ]


In [4]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(80, 4)
(20, 4)
(80,)
(20,)


In [5]:
class SVM:
    def __init__(self, max_iter=100, kernel='linear'，epsilon=0.0001):
        self.max_iter=max_iter
        self.kernel_type = kernel
        self.epsilon = epsilon

    def init_args(self, features, labels):
        self.m, self.n = features.shape
        #print(self.m)
        self.X = features
        self.Y = labels
        self.b = 0.
        #print(self.m)
        #print(self.Y)

        self.alpha = np.ones(self.m)

        self.E_i = [self.E(i) for i in range(self.m)]     #list
        print(len(self.E_i))
        # 松弛变量 C
        self.C = 1.0

    # g(x) 预测值，输入
    def g(self, i):
        r = self.b
        for j in range(self.m):
            r += self.alpha[j]*self.Y[j]*self.kernel(self.X[i], self.X[j])
        return r

    def KKT(self, i):
        Y_g = self.g(i)*self.Y[i]
        if self.alpha[i] == 0:  #if self.alpha[i] < self.epsilon
            return Y_g >=1
        elif 0 < self.alpha[i] and self.alpha[i] < self.C:   #elif self.epsilon < self.alpha[i] and self.alpha[i] < self.C
            return Y_g ==1
        else:
            return Y_g <= 1


    def kernel(self,x1,x2):
        if self.kernel_type == 'linear':
            return sum([x1[k]*x2[k] for k in range(self.n)])
        elif self.kernel_type == 'poly':
            return (sum([x1[k]*x2[k] for k in range(self.n)]) + 1)**2

        return 0

    def E(self, i):
        return self.g(i) - self.Y[i]

    def init_alpha(self):
        # 外层循环先遍历所有满足 0<a<C 的样本点，检验是否满足 KKT
        index_list = [i for i in range(self.m) if 0 < self.alpha[i] < self.C]  
        #index_list = [i for i in range(self.m) if self.epsilon < self.alpha[i] < self.C]
        # 否则遍历整个训练集
        non_satisfy_list = [i for i in range(self.m) if i not in index_list]
        index_list.extend(non_satisfy_list)

        for i in index_list:
            if self.KKT(i):
                continue
            E1 = self.E_i[i]

            # 如果 E1 是正的，选择最小的Ei，如果 E1 是负的，选择最大的Ei
            if E1 >= 0:
                j = min(range(self.m), key=lambda x: self.E_i[x])
            else:
                j = max(range(self.m), key=lambda x: self.E_i[x])
            return i, j

    def compare(self, alpha, L, H):
        if alpha > H:
            return H
        elif alpha < L:
            return L
        else:
            return alpha

    def fit(self, features, labels):
        self.init_args(features, labels)

        for t in range(self.max_iter):
            # train
            i1, i2 = self.init_alpha()

            # boundary
            if self.Y[i1] == self.Y[i2]:
                L = max(0, self.alpha[i1] + self.alpha[i2] - self.C)
                H = min(self.C, self.alpha[i1] + self.alpha[i2])
            else:
                L = max(0, self.alpha[i2] - self.alpha[i1])
                H = min(self.C, self.C + self.alpha[i2] - self.alpha[i1])

            E1 = self.E_i[i1]
            E2 = self.E_i[i2]
            # eta = K11 + K22 - 2K12
            eta = self.kernel(self.X[i1], self.X[i1]) + self.kernel(self.X[i2], self.X[i2])\
                   - 2 * self.kernel(self.X[i1], self.X[i2])
            #print(eta)
            if eta <= 0:
                continue

            alpha2_new_unc = self.alpha[i2] + self.Y[i2] * (E2-E1)/eta
            alpha2_new = self.compare(alpha2_new_unc, L, H)

            alpha1_new = self.alpha[i1] + self.Y[i1] * self.Y[i2]*(self.alpha[i2] - alpha2_new)

            b1_new = -E1 - self.Y[i1] * self.kernel(self.X[i1], self.X[i1])*(alpha1_new-self.alpha[i1])-\
                self.Y[i2] * self.kernel(self.X[i2], self.X[i1]) * (alpha2_new - self.alpha[i2]) + self.b
            b2_new = -E2 - self.Y[i1] * self.kernel(self.X[i1], self.X[i2]) * (alpha1_new-self.alpha[i1]) - \
                     self.Y[i2] * self.kernel(self.X[i2], self.X[i2]) * (alpha2_new-self.alpha[i2]) + self.b

            if 0 < alpha1_new < self.C:
                b_new = b1_new
            elif 0 < alpha2_new < self.C:
                b_new = b2_new
            else:
                # 选择中点
                b_new = (b1_new+b2_new)/2

            # 更新参数
            self.alpha[i1] = alpha1_new
            self.alpha[i2] = alpha2_new
            self.b = b_new

            self.E_i[i1] = self.E(i1)
            self.E_i[i2] = self.E(i2)

        return "Done ! "

    def predict(self, data):
        r = self.b
        for i in range(self.m):
            r += self.alpha[i] * self.Y[i] * self.kernel(data, self.X[i])

        return 1 if r > 0 else -1

    def score(self, X_test, y_test):
        right_count = 0
        for i in range(len(X_test)):
            result = self.predict(X_test[i])
            if result == y_test[i]:
                right_count += 1
        return right_count / len(X_test)

SyntaxError: invalid character in identifier (<ipython-input-5-e196c4e63124>, line 2)

In [ ]:
svm = SVM(max_iter=200)
print(svm.fit(X_train, y_train))
accu = svm.score(X_test, y_test)
print(f"Accuracy: {accu:.4f}")